# Post Scrapping


## Introduction : 

**How to use :** 

installer python 3.10 
ne pas oublier de le mettre dans les variables d'environnement

crée un environnement python ```python -m venv venv```

activer l'environnement ```venv\Scripts\Activate```

Installer les packages ```pip install -r --no-cache-dir requirements.txt```

Vous êtes a travailler 

## Script : 

In [1]:
import pandas as pd
import re
import json
import urllib3
from appwrite.client import Client
from appwrite.services.databases import Databases
from appwrite.id import ID
from appwrite.query import Query

from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
client = Client()

client.set_endpoint("https://cloud.appwrite.io/v1")  # Your API Endpoint
client.set_project("onmangeou")  # Your project ID
client.set_key(os.getenv("apiKeyAppWrite"))  # Your secret API key


databases = Databases(client)


priceRef = databases.list_documents(
    database_id=os.getenv("databaseId"), collection_id=os.getenv("tableRestaurantPrice")
)["documents"]

## Tairement de la donnée

In [3]:
df = pd.read_json("data.json")
df["name"] = df["name"].apply(lambda x: x[:49])
output = df[["index","name","gmapLink"]].copy()
output.set_index("index",inplace=True)
df.set_index("index", inplace=True)
df.shape

(5, 11)

In [4]:
def getWebsite(elem):
    url = re.findall(r">[a-zA-Z]*\.[a-zA-Z]*<", elem)
    if len(url)>0 : 
        url = url[0][1:-1]
        if url.find("https://") == -1 : 
            url = "https://" + url
    else : 
        url = "https://www.google.fr/maps"
    return url

def getPhone(elem):
    regex = re.search(r"([0-9]{1,3} ){4}[0-9]{1,3}", elem)
    if regex is not None:
        phone = regex.group().replace(" ","")
        return phone

def getPriceId(price):
    if price is None or type(price) == float:
        id = list(filter(lambda x: x["euroConvert"] == 1, priceRef))
    else:
        id = list(filter(lambda x: x["euroConvert"] == len(price), priceRef))
    return id[0]["$id"] 

In [5]:
def gethours(elem):
    if elem is not None:
        formatedHours = []
        for hoursStr in elem:
            splited = hoursStr.split(",")
            for hourStamp in splited[1:]:
                hours = re.findall(r"[0-9]{2}:[0-9]{2}", hourStamp)
                if not hours == []:
                    formatedHours.append(
                            {"openingTime": hours[0], "closingTime": hours[1], "dayOfWeek": splited[0]}
                        )


        return formatedHours

def parseAbout(elem):
    returnDict = []
    for key, values in elem.items():
        valideValues = list(filter(lambda x: x[1].find("check_black_18dp") > 1, values))
        if len(valideValues) > 0:
            returnDict.append({"name":key,"details":[v[0] for v in valideValues]})
    return returnDict



In [6]:
df["website"] = df["info"].apply(getWebsite)
df["coordonne"] = df["gmapLink"].apply(
    lambda x: re.findall(r"[0-9]{1,3}\.[0-9]*!4d[0-9]{1,3}\.[0-9]*", x)[0].split("!4d")
)

In [11]:
# df["lenPrice"] = df["price"].apply(lambda x: len(x) if x is not None else 0)
df["googleMapRating"] = df["googleMapRating"].apply(
    lambda x: float(x.split("(")[0].replace(",", "."))
)
df["address"] = df["address"].apply(lambda x: x.replace("", ""))

In [12]:

output["image"] = df["image"]
output["googleMapRating"] = df["googleMapRating"] 
output["price"] = df["price"].apply(getPriceId)
output["address"] = df["address"]
output["restaurantTypes"] = df["restaurantTypes"].apply(lambda x: [{"name":x}])
output["restaurantService"] = df["restaurantService"].apply(parseAbout)
output["website"] = df["info"].apply(getWebsite)
output["lat"] = df["coordonne"].apply(lambda x: float(x[0]))
output["long"] = df["coordonne"].apply(lambda x: float(x[1]))
output["phone"] = df["info"].apply(getPhone)
output["restaurantHours"] = df["heures"].apply(gethours)

## AppWrite

In [13]:
data = output.to_dict(orient="records")
retour = []
for elem in data:
    searchExisting = databases.list_documents(
        database_id=os.getenv("databaseId"),
        collection_id=os.getenv("tableRestaurant"),
        queries=[
            Query.equal("lat", elem["lat"]),
            Query.equal("long", elem["long"]),
        ],
    )["documents"]
    
    if not len(searchExisting) > 0:
        print("allready exist", len(searchExisting) > 0)
        retour.append(elem)
        iddocs = ID.unique()
        r = databases.create_document(
            database_id=os.getenv("databaseId"),
            collection_id=os.getenv("tableRestaurant"),
            document_id=iddocs,
            data=elem,
        )



allready exist False
allready exist False
allready exist False
allready exist False
allready exist False


In [ ]:
retour[-1]

{'name': 'Itsy Bitsy',
 'gmapLink': 'https://www.google.fr/maps/place/Itsy+Bitsy/data=!4m7!3m6!1s0x47c2d584ad61e47b:0x5ed0698c8d0522fe!8m2!3d50.6317613!4d3.0629843!16s%2Fg%2F11g0h1z9zb!19sChIJe-RhrYTVwkcR_iIFjYxp0F4?authuser=0&hl=fr&rclk=1',
 'image': 'https://lh5.googleusercontent.com/p/AF1QipOyRyITAQaa74d8LQ-Z6CMJC2rxV7RxEDTUK-KS=w408-h306-k-no',
 'googleMapRating': 4.7,
 'price': '66608c5e000709492e6c',
 'address': '10 Rue Jeanne Maillotte, 59800 Lille',
 'restaurantTypes': [{'name': 'Restaurant végétalien'}],
 'restaurantService': [{'name': 'Accessibilité',
   'details': ['Places assises accessibles en fauteuil roulant',
    'Toilettes accessibles en fauteuil roulant']},
  {'name': 'Services disponibles',
   'details': ['Terrasse', 'Vente à emporter', 'Repas sur place']},
  {'name': 'Offre',
   'details': ['Alcools',
    'Bière',
    'Cafés',
    'Convient aux végétariens',
    'Petites portions à partager',
    'Plats végétaliens',
    'Produits sains',
    'Vin']},
  {'name': 'Se

In [ ]:
retour[

SyntaxError: '[' was never closed (3628723250.py, line 1)